In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root= 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

100%|██████████| 26.4M/26.4M [00:10<00:00, 2.44MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 227kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 2.43MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 41.0MB/s]


In [3]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X,y in test_dataloader:
    print(f"Shape of Input = {X.shape} and shape of target y = {y.shape}")

Shape of Input = torch.Size([64, 1, 28, 28]) and shape of target y = torch.Size([64])
Shape of Input = torch.Size([64, 1, 28, 28]) and shape of target y = torch.Size([64])
Shape of Input = torch.Size([64, 1, 28, 28]) and shape of target y = torch.Size([64])
Shape of Input = torch.Size([64, 1, 28, 28]) and shape of target y = torch.Size([64])
Shape of Input = torch.Size([64, 1, 28, 28]) and shape of target y = torch.Size([64])
Shape of Input = torch.Size([64, 1, 28, 28]) and shape of target y = torch.Size([64])
Shape of Input = torch.Size([64, 1, 28, 28]) and shape of target y = torch.Size([64])
Shape of Input = torch.Size([64, 1, 28, 28]) and shape of target y = torch.Size([64])
Shape of Input = torch.Size([64, 1, 28, 28]) and shape of target y = torch.Size([64])
Shape of Input = torch.Size([64, 1, 28, 28]) and shape of target y = torch.Size([64])
Shape of Input = torch.Size([64, 1, 28, 28]) and shape of target y = torch.Size([64])
Shape of Input = torch.Size([64, 1, 28, 28]) and shape

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device {device}")

Using device cuda


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return x

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        preds = model(X)
        loss = loss_fn(preds, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (batch % 100 == 0):
            loss_curr, current = loss.item(), (batch+1)*len(X)
            print(f"Loss = {loss_curr}, currently at : [{current:>5d}/{size:>5d}]")
            

In [13]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    for batch,(X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        preds = model(X)
        test_loss += loss_fn(preds, y).item()
        correct += (preds.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= num_batches

    print(f"Test accuract = {correct} and test loss = {test_loss}")


In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [14]:
epochs = 5
for t in range(epochs):
    print(f"Epoch: {t+1}\n------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!")

Epoch: 1
------------------------------
Loss = 1.8023966550827026, currently at : [   64/60000]
Loss = 1.8236957788467407, currently at : [ 6464/60000]
Loss = 1.8312690258026123, currently at : [12864/60000]
Loss = 1.8028963804244995, currently at : [19264/60000]
Loss = 1.7261391878128052, currently at : [25664/60000]
Loss = 1.713293194770813, currently at : [32064/60000]
Loss = 1.6153388023376465, currently at : [38464/60000]
Loss = 1.5810120105743408, currently at : [44864/60000]
Loss = 1.459591269493103, currently at : [51264/60000]
Loss = 1.4647800922393799, currently at : [57664/60000]
Test accuract = 37.92993630573248 and test loss = 1.5027998010064387
Epoch: 2
------------------------------
Loss = 1.4542059898376465, currently at : [   64/60000]
Loss = 1.4354515075683594, currently at : [ 6464/60000]
Loss = 1.5222896337509155, currently at : [12864/60000]
Loss = 1.4752684831619263, currently at : [19264/60000]
Loss = 1.336081862449646, currently at : [25664/60000]
Loss = 1.29362